# Verify Source and Target Databases and the Ingestion Pipeline <a name="top"></a>

## Table of Contents:

1. [Overview](#Overview)
2. [Aurora MySQL as Source Database](#Aurora-MySQL-as-Source-Database)
3. [Amazon Redshift as Data Warehouse](#Amazon-Redshift-as-Data-Warehouse)
4. [AWS DMS as Near Real-Time Ingestion Pipeline](#AWS-DMS-as-Near-Real-Time-Ingestion-Pipeline)
5. [Simulate Inserts](#Simulate-Inserts)

## Overview
[(Back to the top)](#top)

We will use this notebook to verify that our Aurora MySQL and Amazon Redshift Databases are up and running. 

The Aurora MySQL database will serve as the source of transactions, and the Amazon Redshift database will serve as the target Data Warehouse. We will execute inserts from this notebook as well to simulate new transactions.

<img src="../resources/module2_architecture_diagram.png" alt="Module2 Architecture Diagram]" style="width: 1000px;"/>



## Aurora MySQL as Source Database
[(Back to the top)](#top)
    
Let's first test connectivity to our database:

In [ ]:
import MySQLdb,random,time
from tqdm.notebook import trange, tqdm

host = '###mysql_host###'
user = 'master'
password = 'S3cretPwd99'
port = 3306
db = 'salesdb'

conn = MySQLdb.Connection(
    host=host,
    user=user,
    passwd=password,
    port=port,
    db=db
)

Let's run some SQL statements. We will use the following helper functions to execute SQL statements:

In [ ]:
def execute_sql(sql):
    results=[]
    conn.query(sql)
    result = conn.store_result()
    for i in range(result.num_rows()):
        r = result.fetch_row()
        #print(r)
        results.append(r)
    return results
        
def execute_dml(sql):
    conn.query(sql)
    rowcount = conn.affected_rows()
    print ("Rows updated: %d"%rowcount)
    conn.commit()

In [ ]:
execute_sql("show tables")

This is a generic SALES OLTP schema. Of the tables above, the SALES_ORDER_DETAIL is the one for which we will be inserting records.

## Amazon Redshift as Data Warehouse
[(Back to the top)](#top)

Let's test connectivity to our target datawarehouse:

In [ ]:
import psycopg2
import warnings
warnings.filterwarnings('ignore')

rs_host='###redshift_host###'
rs_dbname='sales_analytics_dw'
rs_user = 'awsuser'
rs_password = 'S3cretPwd99'
rs_port = 5439

con=psycopg2.connect(dbname=rs_dbname, host=rs_host, port=rs_port, user=rs_user, password=rs_password)


Now that the connectivity to the Amazon Redshift database is working fine, let's load the schema to the Amazon Redshift database:

In [ ]:
def load_redshift_schemas(conn, scriptFileName):
    with open (scriptFileName, "r") as scriptfile:
        contents=scriptfile.read()
    commands=[]
    for sql in str(contents).split(";")[:-1]:
        commands.append(sql)
    for sql in commands:
        cursor = con.cursor()
        cursor.execute(sql)
        cursor.close()
        # commit the changes
        con.commit()
    cursor = con.cursor()
    cursor.execute("Select distinct tablename from PG_TABLE_DEF where schemaname = 'public'")
    rows = cursor.fetchall()
    for row in rows:
        print ("   ", row)
    cursor.close()

load_redshift_schemas(con,'redshift-schema.sql')

## AWS DMS as Near Real-Time Ingestion Pipeline
[(Back to the top)](#top)

In this step we will execute a full load of data from this database to Amazon S3 using AWS DMS:

- Navigate to the DMS Console by clicking on Services -> DMS. 
- Locate the menu item Conversion & migration->Database migration tasks from the left-hand panel of the DMS Console.
- Select the only Replication Task item and click on the button Actions -> Restart/Resume to start this task.
- You can monitor the progress of this task by clicking on the task link and viewing the 'Table Statistics' tab. 

## Simulate Inserts
[(Back to the top)](#top)

Let's perform some Inserts to our data. We will use the helper function below to perform the inserts.

In [ ]:
def insert_orders(order_id,new_order_id,max_line_id):
    print (new_order_id)
    execute_dml(f"insert into SALES_ORDER( ORDER_ID, SITE_ID,ORDER_DATE,SHIP_MODE ) select {new_order_id},  SITE_ID,ORDER_DATE,SHIP_MODE from SALES_ORDER where ORDER_ID={order_id}")
    execute_dml(f"insert into SALES_ORDER_DETAIL( ORDER_ID, LINE_ID,LINE_NUMBER,PRODUCT_ID,QUANTITY,UNIT_PRICE,DISCOUNT,SUPPLY_COST,TAX,ORDER_DATE ) select {new_order_id}, {max_line_id}+LINE_ID,LINE_NUMBER,PRODUCT_ID,QUANTITY,UNIT_PRICE,DISCOUNT,SUPPLY_COST,TAX,ORDER_DATE from SALES_ORDER_DETAIL where ORDER_ID={order_id}")   
    
def generate_orders(n):
    new_order_id=execute_sql('select max(order_id) FROM SALES_ORDER')[0][0][0]
    max_line_id=execute_sql('select max(line_id) FROM SALES_ORDER_DETAIL')[0][0][0]
    print (f"max_line_id : {max_line_id}")
    for i in tqdm(range(n)):
        order_id=random.randint(1,29000)
        new_order_id +=1
        insert_orders(order_id,new_order_id,max_line_id)

In [ ]:
generate_orders(100)

We can view the updates that AWS DMS has pushed through using the 'Table Statistics' tab for the Replication task within the AWS DMS Console.

<div class="alert alert-block alert-info"><b>Note:</b> Please keep this notebook open as we move to the 2nd notebook in this Module to execute the AWS Glue incremental ETL jobs. We will execute the cell above again when we need to insert more data.</div>